In [4]:
import sys
sys.path.append('../')
from utilities.task_service import get_authentication_token, get_task, send_answer
from auth import apikey
import re 
import requests

from openai import OpenAI

from dotenv import load_dotenv
load_dotenv(override=True)

task_name = "whisper"

In [5]:
# autoryzacja

token = get_authentication_token(apikey, task_name)
if token:
    print("Otrzymano token.")
else:
    print("Nie udało się uzyskać tokenu.")

Otrzymano token.


In [6]:
# pobranie danych do zadania

response = get_task(token)
if response:
    print("Odpowiedź z serwera:", response)
else:
    print("Wystąpił błąd podczas wysyłania zapytania.")

Odpowiedź z serwera: {'code': 0, 'msg': 'please return transcription of this file: https://tasks.aidevs.pl/data/mateusz.mp3', 'hint': 'use WHISPER model - https://platform.openai.com/docs/guides/speech-to-text'}


In [7]:
# Wyodrębnienie linku do nagrania za pomocą wyrażenia regularnego
link_match = re.search(r"https?://[^\s]+", response['msg'])
if link_match:
    url = link_match.group()
    print(f"Wyodrębniony link: {url}")
else:
    print("W tekście nie znaleziono linku.")


# pobranie pliku (pewnie można to zrobić bez pobierania?)
def download_mp3(url, output_path):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            with open(output_path, 'wb') as f:
                f.write(response.content)
            print(f"MP3 file downloaded successfully to {output_path}")
        else:
            print(f"Failed to download MP3 file. Status code: {response.status_code}")
    except Exception as e:
        print(f"Error occurred while downloading MP3 file: {e}")

output_file = "task-audio.mp3"
download_mp3(url, output_file)

Wyodrębniony link: https://tasks.aidevs.pl/data/mateusz.mp3
MP3 file downloaded successfully to task-audio.mp3


In [8]:
client = OpenAI()

audio_file= open("task-audio.mp3", "rb")
transcription = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file
)
print(transcription.text)

Cześć! Kiedy ostatnio korzystaliście z sztucznej inteligencji, czy zastanawialiście się nad tym, skąd czerpie ona swoją wiedzę? No pewnie, że tak, inaczej nie byłoby was tutaj na szkoleniu. Ale czy przemyśleliście możliwość dostosowania tej wiedzy do waszych własnych, indywidualnych potrzeb?


In [9]:
# wysłanie odpowiedzi

response = send_answer(token, transcription.text)
if response:
    print("Odpowiedź z serwera:", response)
else:
    print("Wystąpił błąd podczas wysyłania odpowiedzi.")

Odpowiedź z serwera: {'code': 0, 'msg': 'OK', 'note': 'CORRECT'}
